<a href="https://colab.research.google.com/github/JohnsonYu0924/114_2_text-analysis/blob/main/%E6%9C%9F%E6%9C%AB%E5%A0%B1%E5%91%8A/script/%E8%99%95%E7%90%86%E9%80%90%E5%AD%97%E7%A8%BF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STEP 0：環境設定

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import json, re, os, csv
import re

os.getcwd()

'/content'

# STEP 1：抓取逐字稿

In [ ]:
# 先寫看看

with open('191218_政見發表會.md', 'r', encoding='utf-8') as t:
    transcript = t.read()
    print(type(transcript)) #class

senten = []
transcript = re.split(r"\n", transcript)
for i in transcript:
    if i != "":
        senten.append(i.strip())

candidate = input("要查詢的人：")
target_candidate = '#### ' + candidate
candidateTranscript = []
is_collecting = False

for front, later in zip(senten[:-1], senten[1:]):
  if front == target_candidate:
    is_collecting = not is_collecting
  if is_collecting and not re.search(r"###", later):
    candidateTranscript.append(later)
    # print(later)
  elif is_collecting and re.search(r"###", later):
    is_collecting = not is_collecting

for i in candidateTranscript:
  print(i)


In [ ]:
# 請Gemini幫我變成def好處理

import re
import os

def candiTrans(filename, candidate):
    """
    從指定的 Markdown 檔案中提取特定候選人的發言內容。

    發言區段的判斷邏輯是：
    1. 遇到 '#### [候選人姓名]' 標記，開啟/關閉收集狀態 (is_collecting)。
    2. 在收集狀態下，直到下一個包含 '###' 的行 (代表另一位候選人發言的開始)
       出現為止，收集該區間的所有內容。

    Args:
        filename (str): 要讀取的檔案名稱。
        candidate (str): 要查詢的候選人姓名 (例如: '韓國瑜' 或 '蔡英文')。

    Returns:
        list: 包含該候選人所有發言句子的列表。
    """

    # 檢查檔案是否存在
    if not os.path.exists(filename):
        print(f"錯誤：檔案 '{filename}' 不存在。")
        return []

    # 1. 檔案讀取與文本清洗
    try:
        with open(filename, 'r', encoding='utf-8') as t:
            transcript = t.read()
    except Exception as e:
        print(f"讀取檔案時發生錯誤: {e}")
        return []

    # 將文本按換行符分割並清除空白行和行首尾空白
    transcript_lines = re.split(r"\n", transcript)
    senten = []
    for line in transcript_lines:
        stripped_line = line.strip()
        if stripped_line != "":
            senten.append(stripped_line)

    # 2. 設定目標分隔符號
    target_candidate = '#### ' + candidate
    candidateTranscript = []
    is_collecting = False

    # 3. 遍歷並提取發言內容 (使用 zip 進行前/後配對)
    # 注意: 這個邏輯不包含區段開頭和結尾的分隔符號本身。
    # 並且，這裡假設所有分隔符號都至少包含 '###'。
    for front, later in zip(senten[:-1], senten[1:]):

        # 檢查是否遇到目標候選人發言的開頭 (例如: '#### 韓國瑜')
        if front == target_candidate:
            # 切換狀態：False -> True (開始收集) 或 True -> False (結束收集)
            is_collecting = not is_collecting

        # 如果正在收集且下一句不是分隔符號
        if is_collecting and not re.search(r"###", later):
            candidateTranscript.append(later)

        # 如果正在收集且下一句是分隔符號 (結束收集)
        elif is_collecting and re.search(r"###", later):
            # 切換狀態：True -> False (停止收集)
            is_collecting = not is_collecting

    return candidateTranscript

In [ ]:
# 調用function

file_name = input("請輸入檔案名稱：")
file_name = file_name + '.md'

candidate_name = input("請輸入要查詢的人：")

result_list = candiTrans(file_name, candidate_name)

if result_list:
    for item in result_list:
        print(item)
else:
    print("未找到發言內容。")


In [ ]:
jsonDataName = input('輸入檔名（eg. 191218_Tsai/Han/Soong): ')
jsonDataName = jsonDataName + ".json"

with open(jsonDataName, 'w', encoding='utf-8') as d:
  json.dump(result_list, d, ensure_ascii=False, indent=4)

#重複做把所有.md檔整理成.json

輸入檔名（eg. 191218_Tsai/Han/Soong): 191229_Soong


# STEP 4：清理逐字稿

In [ ]:
def basicCleaner(file_name):
  # file_name = '191227_Han.json'
  file_name_w = re.sub(r"\.json", "", file_name)

  with open(file_name, 'r', encoding='utf-8') as f:
    transcript = json.load(f)
    print(transcript)

  cleanTrans = []
  for i in transcript:
    i = re.sub(r"https://\S+", " ", i)
    i = re.sub(r"[!\[\]:「」『』《》〈〉（）\(\)，、：；。\.]", " ", i)
    # i = re.sub(r"\d+", "", i) #可以考慮要不要處理數字，跟情緒沒什麼關係
    i = i.strip().lower()
    cleanTrans.append(i)

  for i in cleanTrans:
    print(i)

  file_name_w = file_name_w + '_cleaned.json'
  with open(file_name_w, 'w', encoding='utf-8') as f:
    json.dump(cleanTrans, f, ensure_ascii=False, indent=4)


In [ ]:
# 調用function批次處理

path = "/content/"
print(os.getcwd())
dirList = [i for i in os.listdir(path) if i.endswith(".json")]
if len(dirList) == 0:
  print("There's no '.json' data.")
else:
  print(dirList)

for i in dirList:
  basicCleaner(i)

# STEP 5：切token
利用中研院「CKIP Transformers」代替「jieba」進行token處理

In [22]:
# !pip install -U ckip-transformers

# import tools: token(WS), tagging(POS), 實體辨識(NER)
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker

# diver initiation(初始化一次之後先關一下)
WS_driver = CkipWordSegmenter(model='bert-base')
POS_driver = CkipPosTagger(model='bert-base')
NER_driver = CkipNerChunker(model='bert-base')

In [28]:
# try...
# imput data
import os, json
with open("/content/cleaned/191218_Han_cleaned.json", "r", encoding="utf-8") as f:
  myText = json.load(f)
  myText = myText[:10]
  print(myText)

# run
myText_WS = WS_driver(myText)
# # myText_POS = POS_driver(myText_WS)
# # myText_NER = NER_driver(myText)

# # print result
print("\nWS \n", myText_WS)
# print("POS \n", myText_POS)
# print("NER \n", myText_NER)

['謝謝主持人李進勇主委 蔡英文蔡總統 宋楚瑜宋主席 全國各位所有的台灣同胞 各位海內外關切中華民國第 15 任總統選舉的所有的好朋友大家晚安 大家好！！', '首先非常感謝大家這麼長的一段時間 對韓國瑜的愛護以及支持！更要感謝高雄市 280 萬市民朋友選我為高雄市長 一路陪著我走上這一段神奇的旅程 讓我有這個機會跟大家一起來做出對中華民國未來的改變 以及我們共同來創造未來', '今天是公辦政見發表 所有的政見國防 內政 外交 交通 青年人的未來 毒品治安等等非常非常的多 國家要做的事情非常多 但是這麼多事情之前 有一個最重要的就是我們到底是什麼國家？我們現在到底在辯論什麼？我們是中華民國第 15 任總統的政見發表 還是 這個國家 的政見發表？蔡英文總統擔任總統三年半以來 從來沒有對國家講得清楚 前三年用 這個國家 來代替 所以今天我們政見發表是不是應該改名為 這個國家 政見發表會 還是中華民國總統的政見發表會？', '沒有國就不會有家 我長期一直問蔡英文總統到底要帶兩千三百萬同胞何去何從？蔡英文總統身為中華民國總統 享受到中華民國憲法授予的一切權力跟待遇 但是都是用 這個國家 來稱呼 所以請問蔡英文總統如果不把國家定位講清楚 我們後面的 我們政見說明有什麼意義？', '蔡英文總統眼前有三條路去選擇 我們看的非常清楚 第一 你根本就不愛中華民國 第二 你不要九二共識 我們國民黨的立場非常清楚 在中華民國完整的國格現任體制之下 發展兩岸的交流 是九二共識 你一定要扣一個帽子 叫做我們認同一國兩制 第三 我覺得蔡英文總統更講不清楚 而且完全對不起長期支持台灣獨立的朋友 很多台獨的朋友把選票投給蔡英文總統 希望台灣獨立 但是你完全讓他們失望 所以蔡英文總統必須有責任告訴我們台灣同胞 未來兩千三百萬民眾 到底何去何從？', '韓國瑜的立場非常清楚 我就是支持中華民國 捍衛中華民國主權 捍衛中華民國憲政體制 以及服務跟保護兩千三百萬台澎金馬的同胞生命財產 一清二楚 沒有任何含糊空間', '如果蔡英文總統支持台灣獨立 也請大聲地講出來 如果不支持台獨 也請大聲地講出來 這樣子兩千三百萬民眾在投票給你的時候 才知道你要把我們帶到哪裡去 所以我希望蔡總統如果不支持台獨 請講清楚之後請跟著我的方向 我大喊三聲 我希望蔡英文總統你也大喊三聲 中華民國萬歲！中華民國萬歲！中華民國萬歲！希

Inference: 100%|██████████| 1/1 [00:07<00:00,  7.04s/it]


WS 
 [['謝謝', '主持人', '李進勇', '主委', ' ', '蔡英文', '蔡', '總統', ' ', '宋楚瑜', '宋', '主席', ' ', '全', '國', '各位', '所有', '的', '台灣', '同胞', ' ', '各位', '海內外', '關切', '中華民國', '第 15 ', '任', '總統', '選舉', '的', '所有', '的', '好朋友', '大家', '晚安', ' ', '大家', '好', '！', '！'], ['首先', '非常', '感謝', '大家', '這麼', '長', '的', '一', '段', '時間', ' ', '對', '韓國瑜', '的', '愛護', '以及', '支持', '！', '更', '要', '感謝', '高雄市', ' 280', ' ', '萬', '市民', '朋友', '選', '我', '為', '高雄', '市長', ' ', '一路', '陪', '著', '我', '走上', '這', '一', '段', '神奇', '的', '旅程', ' ', '讓', '我', '有', '這', '個', '機會', '跟', '大家', '一起', '來', '做出', '對', '中華民國', '未來', '的', '改變', ' ', '以及', '我們', '共同', '來', '創造', '未來'], ['今天', '是', '公辦', '政見', '發表', ' ', '所有', '的', '政見', '國防', ' ', '內政', ' ', '外交', ' ', '交通', ' ', '青年人', '的', '未來', ' ', '毒品', '治安', '等等', '非常', '非常', '的', '多', ' ', '國家', '要', '做', '的', '事情', '非常', '多', ' ', '但是', '這麼多', '事情', '之前', ' ', '有', '一', '個', '最', '重要', '的', '就', '是', '我們', '到底', '是', '什麼', '國家', '？', '我們', '現在', '到底', '在', '辯論', '什麼', '？', '我們', '是', '中華民國', '第 1

In [40]:
# 利用CKIP來tokenize，並引入stopwords清理一下

# import stopwords
with open("/content/stopwords.txt", "r", encoding="utf-8") as f:
  stopwords = f.read().split("\n")
  stopwords = set(stopwords)

#
def tokener(dataName):
  '''
  利用CKIP進行tokenize,並利用stopwords來清理。
  dataName放入含有.json的檔名就可以開始使用...

  使用前先確認
    (1) ckip WS driver是否已經初始化
    (2) topwords辭典是否有放進工作區
    (3) 路徑是否有正確：os.chdir(path)

  祝一切順利！
  '''
  with open("/content/cleaned/" + dataName, "r", encoding="utf-8") as f:
    myText = json.load(f)

  myText_WS = WS_driver(myText)
  print(myText_WS)

  myText_token = []
  for i in myText_WS:
    for j in i:
      if j not in stopwords:
        myText_token.append(j)
  print(myText_token)


  dataName = re.sub(r"cleaned\.json", "", dataName)
  dataName = dataName + "tokened.json"
  with open("/content/tokened/" + dataName, "w", encoding="utf-8") as f:
    json.dump(myText_token, f, ensure_ascii=False, indent=4)
    os.close

# 批次處理
path = "/content/cleaned"
dirlist = [i for i in os.listdir(path) if i.endswith(".json")]
# print(dirlist)

for data in dirlist:
  tokener(data)

Inference: 100%|██████████| 1/1 [01:59<00:00, 119.54s/it]


[['主持人', ' ', '監察人', ' ', '兩', '位', '候選人', ' ', '海內外', '的', '各位', '父老', '兄弟', '姐妹', ' ', '各位', '鄉親', ' ', '大家', '好'], ['我', '是', '中華民國', '第 15 ', '任', '總統', '候選人', '宋楚瑜'], ['我', '今天', '穿', '這', '身', '服裝', ' ', '請', '蔡', '總統', '不要', '緊張', ' ', '我', '不', '是', '要', '質疑', '您', '的', '博士', '學位', ' ', '而是', '一般', '我們', '國人', '要', '曉得', ' ', '得', '博士', '要', '有', '幾', '個', '很', '重要', '的', '這些', '證明'], ['第一', ' ', '他', '要', '完成', '所有', '的', '這些', '課程', ' ', '第二', ' ', '他', '要', '通過', '博士', '的', '這些', '事項', ' ', '最', '重要', '的', '考試', ' ', '不管', '是', '語文', '也好', ' ', '或者', '是', '他', '的', '專業', '科目', ' ', '我', '是', '政治學', '博士', ' ', '我', '要', '通過', '政治學', '理論', ' ', '美國', '政府', ' ', '各', '國', '政府', ' ', '以及', '美國', '政府', '的', '這些', '許許多多', '的', '課程', ' ', '我', '有', '證明'], ['第二', ' ', '我', '必須', '要', '寫', '我', '的', '博士', '論文', ' ', '而', '這', '個', '博士', '論文', '必須', '要', '經過', '老師', '的', '口試', ' ', '我', '不但', '有', '這些', '證明', ' ', '而且', '還', '得到', '優異', '的', '評比', ' ', '也', '有', '證明'], ['第三', ' ', '我'

Inference: 100%|██████████| 1/1 [01:11<00:00, 71.94s/it]


[['主持人', ' ', '監察人', ' ', '還', '有', '親民黨', '的', '候選人', '宋', '先生', ' ', '國民黨', '的', '候選人', '韓國瑜', '先生', ' ', '電視機', '前面', '的', '觀眾', '朋友', ' ', '還', '有', '我們', '在', '線上', '看', '直', '播', '的', '朋友', '們', ' ', '全體', '的', '國人', '同胞', ' ', '大家', '晚安'], ['剛才', '我', '很', '仔細', '地', '聆聽', '了', '韓國瑜', '先生', '的', '發言', ' ', '我', '也', '聽到', '他', '對', '我', '很多', '的', '指教', '跟', '批評', ' ', '但是', '在', '過去', '這', '段', '時間', '他', '跟', '他', '的', '政黨', '也', '對', '我', '很多', '地', '批評', '跟', '指教', ' ', '不過', '仔細', '聽', '一', '聽', ' ', '這些', '發言', '不', '脫', '是', '歧視性', '的', '發言', ' ', '情緒性', '的', '發言', ' ', '或者', '挑起', '社會', '對立', ' ', '甚而', '至於', '是', '沒有', '根據', '的', '指控', ' ', '有時候', '還', '無中生有', ' ', '斷章取義'], ['那', '剛才', '他', '提', '的', '幾', '個', '問題', ' ', '比如說', '中華民國', ' ', '我們', '現在', '不', '就', '是', '正在', '選', '中華民國', '的', '總統', '嗎', '？', '我', '現在', '就', '是', '第', ' 14', ' ', '任', '總統', ' ', '我們', '現在', '在', '選', '第', ' 15 ', '任', '的', '總統'], ['另外', ' ', '我們', '這', '滿', '場', '不', '就', '是', '國旗', '嗎', '

Inference: 100%|██████████| 1/1 [01:01<00:00, 61.87s/it]


...

[['主持人', ' ', '監察人', ' ', '兩', '位', '候選人', ' ', '海內外', '的', '同胞', ' ', '各位', '鄉親', ' ', '大家', '好'], ['今天', '是', '耶誕節', ' ', '也', '是', '中華民國', '行憲', '紀念日', ' ', '首先', '祝福', '各位', '鄉親', '耶誕節', '快樂', ' ', '也', '祝福', '我們', '中華民國', '行憲', '順利'], ['我們', '剛剛', '聽到', '這', '兩', '位', '候選人', '的', '談話', ' ', '我', '就', '想到', '上', '一', '次', '在', '政見', '發表會', '的', '時候', ' ', '我', '特別', '提醒', '我們', '台灣', '要', '注意', '全', '世界', ' 2020', ' ', '年', '將', '有', '重大', '變化'], ['四', '個', '最', '重要', '的', '發生', '的', '事情', '會', '發生', ' ', '簡言之', ' ', '就', '是', '中國', ' ', '日本', ' ', '美國', ' ', '這', '三', '個', '世界', '的', '超強', ' ', '再', '合縱', '連橫', ' ', '而', '世界', '的', '重大', '的', '這些', '變化', '將', '會', '對', '台灣', '有', '什麼', '影響', '？'], ['聽到', '剛剛', '他們', '兩', '位', '的', '談話', ' ', '兩', '個', '重點', ' ', '那', '就', '是', '蔡', '總統', '特別', '談到', '了', ' ', '台灣', '的', '經濟', ' ', '她', '的', '政績', '不錯', ' ', '但是', '我', '特別', '要', '提出來', '的', ' ', '我們', '必須', '要', '從', '高', '來', '看', ' ', '這', '三', '個', '重要', '國家', '的', '互動', ' ', '

Inference: 100%|██████████| 1/1 [00:59<00:00, 59.63s/it]


[['主持人', ' ', '監察人', ' ', '兩', '位', '候選人', ' ', '各位', '海內外', '的', '同胞', ' ', '各位', '鄉親', ' ', '大家', '好', ' ', '我', '是', '中華民國', '第 15', ' ', '任', '總統', '候選人', '宋楚瑜'], ['我', '首先', '應該', '向', '諸', '位', '報告', ' ', '我', '為什麼', '參選', '？'], ['我', '對', '今天', '中華民國', '在', '台', '所', '面臨', '的', '國內外', '環境', '和', '未來', ' ', '我們', '應該', '了解', '得', '到', ' ', '如何', '去', '處理', '很多', '重大', '問題'], ['宋楚瑜', '經歷', ' 44 ', '年', '從政', '經驗', ' ', '我', '追隨', '蔣經國', '先生', ' ', '孫運璿', '先生', ' ', '李登輝', '先生', ' ', '我', '從', '他們', '那裡', '學習到', '很多', '治國', '的', '重大', '的', '絕學', ' ', '我', '想', '在', '我', '有生之年', ' ', '把', '這些', '我', '所', '學到', '的', '經驗', ' ', '奉獻給', '我', '心愛', '的', '國家', '中華民國'], ['我', '自信', '我', '不僅', '有', '經驗', ' ', '有', '能力', '更', '有', '執行力', ' ', '但是', '我', '沒有', '包袱', ' ', '我', '沒有', '私心', ' ', '我', '只有', '包容', '和', '一直', '保持', '從政', '的', '初心', ' ', '那', '就', '是', '腳踏實地', ' ', '認真', '地', '為', '人民', '好好', '地', '處理好', '他們', '希望', '政府', '處理', '的', '事'], ['我', '是', '一', '個', '身體力行', '來', '推動', '民主'

Inference: 100%|██████████| 1/1 [01:26<00:00, 86.56s/it]

[['主持人', ' ', '監察人', ' ', '親民黨', '的', '候選人', '宋', '先生', ' ', '國民黨', '的', '候選人', '韓', '先生', ' ', '我們', '電視機', '前', '的', '觀眾', '朋友', ' ', '還', '有', '我們', '看', '線上', '直', '播', '的', '朋友', ' ', '全體', '國人', '同胞', ' ', '大家', '午安', ' ', '大家', '好'], ['在', '今天', '的', '第一', '輪', '的', '政見會', ' ', '我', '想要', '談', '一下', '台灣', '的', '經濟'], ['在', '上', '週', '的', '政見會', '裡', ' ', '韓', '市長', '曾經', '提到', '世界', '忘記', '了', '台灣', ' ', '那', '我', '有', '提醒', '他', ' ', '事實', '上', '是', '他', '自己', '忘記', '了', '世界', ' ', '世界', '不可能', '忘記', '台灣', ' ', '也', '不會', '忘記', '台灣', ' ', '因為', '台灣', '確實', '是', '一', '個', '很', '重要', '的', '國家'], ['我們', '是', '全', '世界', '第 21 ', '大', '的', '經濟體', ' ', '也', '是', '第', ' 18', ' ', '大', '的', '貿易', '國家', ' ', '國際', '半導體', '供應鍊', '也', '是', '以', '我們', '為', '核心', '的', '基地', ' ', '國際', '經濟', '論壇', '曾經', '肯定', '我們', '是', ' 4 ', '大', '創新', '王國', ' ', '而', '我們', '總體', '經濟', '的', '穩定度', '也', '是', '世界', '第一', '名'], ['台灣', '正在', '大海', '裡面', '乘風破浪', ' ', '勇往直前', ' ', '韓', '市長', '卻', '坐', '在', '浴缸',

# 暫存區

In [ ]:
#刪除資料

import os
import glob

# ⚠️ 請將這裡替換成您要清理的資料夾路徑
folder_path = '/content'

# 定義要刪除的檔名模式
patterns_to_delete = ['*_cleaned.json', '*_cleaned_cleaned.json']

# 遍歷所有模式並刪除檔案
deleted_count = 0
for pattern in patterns_to_delete:
    # 組合完整的搜索路徑
    search_path = os.path.join(folder_path, pattern)

    # glob.glob() 找到所有符合模式的檔案路徑
    for file_path in glob.glob(search_path):
        try:
            # 刪除檔案
            os.remove(file_path)
            print(f"✅ 成功刪除: {file_path}")
            deleted_count += 1
        except OSError as e:
            # 處理刪除失敗的情況（例如權限不足）
            print(f"❌ 錯誤：無法刪除 {file_path}. {e}")

print(f"\n--- 總結：共刪除了 {deleted_count} 個檔案。 ---")

✅ 成功刪除: /content/191227_Soong_cleaned.json
✅ 成功刪除: /content/191218_Tsai_cleaned.json
✅ 成功刪除: /content/191227_Han_cleaned.json
✅ 成功刪除: /content/191229_Han_cleaned.json
✅ 成功刪除: /content/191229_Soong_cleaned.json
✅ 成功刪除: /content/191225_Han_cleaned.json
✅ 成功刪除: /content/191227_Tsai_cleaned.json
✅ 成功刪除: /content/191218_Han_cleaned.json
✅ 成功刪除: /content/191229_Tsai_cleaned.json
✅ 成功刪除: /content/191225_Soong_cleaned.json
✅ 成功刪除: /content/191218_Soong_cleaned.json
✅ 成功刪除: /content/191225_Tsai_cleaned.json

--- 總結：共刪除了 12 個檔案。 ---


## Question: 能不能把上面操作變成批次處理？（好瘋）

In [ ]:
# 從HTHL清資料（先不用這個東西）

HTML1218 = open('1218.html', 'r', encoding='utf-8')
soup = BeautifulSoup(HTML1218, 'html.parser')
# print(soup)


all_spans = soup.find_all('span')
content_list = []
flag = False

for span in all_spans:
    text = span.get_text(strip=True)
    if text == '逐字稿：':
        flag = True
    if flag and text:
        clean_text = re.sub(r'[\u3000\s]+', ' ', text).strip()
        print(text)
    # if clean_text and clean_text != ' ':
    #     content_list.append(clean_text)

FileNotFoundError: [Errno 2] No such file or directory: '1218.html'

In [ ]:
# 列印token和tag的結果

def pack_ws_pos_sentece(sentence_ws, sentence_pos):
    assert len(sentence_ws) == len(sentence_pos)
    res = []
    for word_ws, word_pos in zip(sentence_ws, sentence_pos):
        res.append(f"{word_ws}({word_pos})")
    return "\u3000".join(res)

for myText, ws, pos in zip(myText, myText_WS, myText_POS):
  print(myText)
  print(pack_ws_pos_sentece(ws, pos))

NameError: name 'myText' is not defined